## Install Java

Apache Spark and Sedona (formerly known as GeoSpark) are both built on the Java Virtual Machine (JVM).  This compatibility ensures that they can take advantage of the performance optimizations and garbage collection provided by the JVM.The JVM optimizes Java bytecode at runtime, which can lead to efficient execution of Spark jobs.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

## Download Spark

 Apache Spark is designed for distributed data processing across clusters of computers. Sedona leverages Spark's ability to handle large datasets efficiently by distributing the workload, which is crucial for geospatial data analysis that often involves large volumes of data.

In [2]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz

## Set Environment Variables

This setup is done before starting a Spark application to ensure that the application can locate the Java and Spark installations and access Spark's Python libraries.

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"
os.environ["PYTHONPATH"] = "/content/spark-3.5.3-bin-hadoop3/python"

## Install PySpark

Pyspark provides a Spark-friendly framework that will allow me to leverage Sedona's geospatial capabilities directly on Spark DataFrames. This integration enables efficient processing of large-scale geospatial data without the need to convert data into other formats, such as Pandas.

In [4]:
!pip install findspark
import findspark
findspark.init()

## Install Apache Sedona

Sedona provides a wide range of spatial functions, such as spatial joins, distance calculations, and geometric transformations. These functions are crucial for performing geospatial analyses.

In [5]:
# 1.1
!pip install apache-sedona[spark]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.7/177.7 kB 7.3 MB/s eta 0:00:00


## Start Sedona

#### Imports

Together, these imports facilitate the seamless integration of geospatial data processing into Spark applications, enabling users to leverage the power of distributed computing for large-scale geospatial analyses.

In [6]:
from sedona.register.geo_registrator import SedonaRegistrator
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, when, lit, expr
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import RangeQuery
from datetime import datetime
from shapely.geometry import box
from sedona.core.SpatialRDD import SpatialRDD
from sedona.core.enums import GridType
from sedona.core.geom.envelope import Envelope
from sedona.core.spatialOperator import RangeQueryRaw
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, ArrayType, BooleanType, IntegerType, FloatType, DoubleType

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Config

We set up an Apache Spark environment integrated with Apache Sedona for geospatial data processing. It configures the necessary JAR packages and repositories to ensure that Sedona's geospatial functionalities are available in the Spark session. Finally, it creates a SedonaContext, allowing users to perform spatial analyses on large datasets efficiently

In [8]:
# set up spark which is required
from sedona.spark import *
config = SedonaContext.builder(). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-3.0_2.12:1.6.1,'
           'org.datasyslab:geotools-wrapper:1.6.1-28.2'). \
    config('spark.jars.repositories', 'https://artifacts.unidata.ucar.edu/repository/unidata-all'). \
    getOrCreate()
spark = SedonaContext.create(config)

#### Load Data

In [9]:
# 1.2
data_path = "/content/drive/MyDrive/dbms_dataset.json"
my_df = spark.read.json(data_path)

my_df.printSchema()
my_df.show(truncate=False)

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |

Selecting only the necessary columns helps reduce the size of the DataFrame, making it more manageable for subsequent analyses. This can improve performance by minimizing memory usage and processing time.



In [10]:
# Extract columns which are necessary
my_df = my_df.select("created_at", "coordinates", "text", "place")
my_df.show(10, truncate=False)

+------------------------------+--------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|created_at                    |coordinates                     |text                                                                                                                                                                               |place                                                                                                                                                                                                                                 

This sets the Spark SQL configuration to use the "LEGACY" time parser policy. This is useful when dealing with date and time formats that are not compatible with the newer parsing rules introduced in Spark 3.0.

In [11]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
my_df = my_df.withColumn("created_at", to_timestamp("created_at", "EEE MMM dd HH:mm:ss Z yyyy"))
my_df.show(truncate=False)

+-------------------+-------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|created_at         |coordinates                          |text                                                                                                                                                                               |place                                                                                                                                                                                                                                             

We extract latitude and longitude based on the availability of coordinates and place data. If explicit coordinates are not available, we compute the average longitude and latitude from the bounding box of the place. The resulting values are then used to create a spatial point for further geospatial analysis.

In [12]:
# Extract longitude
my_df = my_df.withColumn("longitude", when(
    col("coordinates").isNull() & col("place").isNotNull(),
    expr("aggregate(place.bounding_box.coordinates[0], cast(0.0 as double), (acc, x) -> acc + cast(x[0] as double)) / 4")
).otherwise(col("coordinates").getField("coordinates")[0]))  # Adjusted access to coordinates

# Extract latitude
my_df = my_df.withColumn("latitude", when(
    col("coordinates").isNull() & col("place").isNotNull(),
    expr("aggregate(place.bounding_box.coordinates[0], cast(0.0 as double), (acc, x) -> acc + cast(x[1] as double)) / 4")
).otherwise(col("coordinates").getField("coordinates")[1]))  # Adjusted access to coordinates

# Create a location column
my_df = my_df.withColumn("location", ST_Point(col("longitude"), col("latitude")))

# Show df
my_df.show(5, truncate=False)


+-------------------+--------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+------------------+--------------------------------------+
|created_at         |coordinates                     |text                                                                                                                                                                               |place                                                                                                                                                                                 

Counting null values is a critical step in the data analysis process. It provides insights into data quality and completeness, informs data cleaning strategies, and helps ensure that the analysis is based on reliable and meaningful data.

In [13]:
null_count_place = my_df.filter(col("place").isNull()).count()
null_count_longitude = my_df.filter(col("longitude").isNull()).count()
null_count_latitude = my_df.filter(col("latitude").isNull()).count()

print(f"Number of null values in 'longitude' column: {null_count_longitude}")
print(f"Number of null values in 'latitude' column: {null_count_latitude}")
print(f"Number of null values in 'place' column: {null_count_place}")

Number of null values in 'longitude' column: 1679
Number of null values in 'latitude' column: 1679
Number of null values in 'place' column: 1914


We do this to clean the DataFrame by removing rows that lack critical information and dropping columns that are no longer needed. This enhances data quality, efficiency, and clarity, which are all crucial for effective data analysis and modeling.






In [14]:
# Drop rows where place value and coordinates are both null
my_df = my_df.filter(~(col("place").isNull() & col("coordinates").isNull()))

# drop place column
my_df = my_df.drop("place", "coordinates")

In [15]:
null_count_location = my_df.filter(col("location").isNull()).count()
print(f"Number of null values in 'location' column: {null_count_location}")

Number of null values in 'location' column: 0


We convert a DataFrame into a Spatial RDD, enabling efficient spatial operations. By analyzing the Spatial RDD, it ensures that the data is ready for spatial indexing, which optimizes spatial queries. The spatial partitioning step organizes the data into a KDB tree structure, improving query performance by reducing the search space. Finally, building an R-tree index enhances the efficiency of spatial queries, allowing for faster retrieval of spatial data based on geometric relationships.


In [16]:
# 1.3
# Convert DataFrame to a Spatial RDD, build rtree (1.3)
spatial_rdd = Adapter.toSpatialRdd(my_df, "location")

spatial_rdd.analyze()

spatial_rdd.spatialPartitioning("KDBTREE")

num_partitions = num_partitions = spatial_rdd.rawSpatialRDD.getNumPartitions()

print(f"Number of partitions: {num_partitions}")

spatial_rdd.buildIndex("RTREE", True)

Number of partitions: 3


We define a specific time range and a bounding box to filter spatial data within a defined geographic area and time period. Then we perform a spatial range query on the Spatial RDD to retrieve relevant records, then extracts and parses user data into a structured format. The parsed data is converted into a DataFrame with a defined schema, enabling further analysis. Finally, it filters the DataFrame to retain only records created within the specified time range, facilitating focused analysis on the desired subset of data.






In [32]:
# 1.4
# Define the time range

t1 = "2017-07-22 00:00:00"
t2 = "2017-07-22 23:59:59"

# Define the bounding box as a geometry object
xmin, ymin, xmax, ymax = -2.3, 53.3, -2.1, 53.6
bounding_box = box(xmin, ymin, xmax, ymax)

# Perform a spatial range query
query_result = RangeQuery.SpatialRangeQuery(
    spatial_rdd,
    bounding_box,
    True,  # Use the index
    False  # Don't use a custom query window
)

result = query_result.collect()

# Extract userdata from the results
user_data_list = [row.getUserData() for row in result]

# Split the string into fields based on tab delimiter
parsed_data = [row.split("\t") for row in user_data_list]

timestamp_format = "%Y-%m-%d %H:%M:%S.%f"

# Iterate over the list and convert the first element to a datetime object
for row in parsed_data:
    row[0] = datetime.strptime(row[0], timestamp_format)

# Check if any element is 'null' and replace it with None
parsed_data = [
    [None if x == 'null' else x for x in row] for row in parsed_data
]

# Define a schema for the resulting DataFrame
schema = ["created_at", "text", "longitude", "latitude"]


# Create the DataFrame from the parsed rows
result_df = spark.createDataFrame(parsed_data, schema)


# Filter data by time
df_filtered_time = result_df.filter(
    (col("created_at") >= lit(t1)) & (col("created_at") <= lit(t2))
)

df_filtered_time.show(truncate=False)

+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|created_at         |text                                                                                                                                             |longitude |latitude          |
+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+
|2017-07-22 09:02:53|@AmandaMullen3 @scottclarke948 Nothing but nothing beats feeling proud ❤❤ I'm made up for you all                                                |-2.23348  |53.4569525        |
|2017-07-22 09:02:57|#shoplocal #supportindependent #cheshire #stockport #poynton @ UBAgene smoke-grill-bakery https://t.co/uIpPkIxGnq                                |-2.1231491|53.3468493        |
|2017-07-2

In [18]:
!pip install hilbertcurve

We implement a Hilbert curve transformation to spatial data, enabling efficient spatial indexing and querying. It first defines functions to calculate Hilbert values based on normalized latitude and longitude coordinates within a specified grid size. The apply_hilbert_curve function processes each row, checking for missing values and computing the corresponding Hilbert value for valid entries. The DataFrame is cleaned to remove rows with null values before mapping the transformation function across the data. The resulting RDD is filtered to exclude any None values and is then converted back into a DataFrame with a defined schema. Finally, the transformed DataFrame is displayed and can be saved to a CSV file for further analysis or use.






In [38]:
# 2.1
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from pyspark.sql import Row
import math

# Helper function to calculate Hilbert value for 2D coordinates (lat, lon)
def hilbert_curve(lat, lon, grid_size=256):
    """
    Given a latitude and longitude, calculate the corresponding Hilbert value
    within a grid of grid_size x grid_size.
    """

    # Normalize the latitude and longitude to the grid's range
    min_lat, max_lat = 53.3, 53.6  # Example bounding box
    min_lon, max_lon = -2.3, -2.1  # Example bounding box

    # Normalize coordinates to [0, grid_size-1]
    x = int((lon - min_lon) / (max_lon - min_lon) * (grid_size - 1))
    y = int((lat - min_lat) / (max_lat - min_lat) * (grid_size - 1))

    # Now calculate the Hilbert value for the grid cell (x, y)
    hilbert_value = xy_to_hilbert(x, y, grid_size)

    return hilbert_value

# Function to convert (x, y) coordinates into Hilbert value
def xy_to_hilbert(x, y, grid_size=256):
    """
    Convert (x, y) coordinates in a grid of size `grid_size` to a Hilbert curve value.
    """
    hilbert_value = 0
    n = grid_size

    for s in reversed(range(int(math.log2(n)))):
        mask = 1 << s
        rx = (x & mask) >> s  # bitwise operation to extract bit for x
        ry = (y & mask) >> s  # bitwise operation to extract bit for y
        hilbert_value = (hilbert_value << 2) | (rx ^ ry)  # Combine bits into hilbert_value
        x, y = rotate(x, y, rx, s)
        x, y = rotate(x, y, ry, s)

    return hilbert_value

# Helper function to rotate coordinates
def rotate(x, y, res, s):
    """
    Rotate the (x, y) coordinates based on the Hilbert curve transformation.
    """
    if res == 0:
        return x, y
    if s % 2 == 0:
        return y, x
    return x, y

# Apply the Hilbert curve transformation on the spatial data
def apply_hilbert_curve(row):
    # Check for missing latitude, longitude or timestamp
    if row["latitude"] is None or row["longitude"] is None or row["created_at"] is None:
        return None

    lat = row["latitude"]
    lon = row["longitude"]
    timestamp = row["created_at"]

    # Compute the Hilbert value for the coordinates
    hilbert_value = hilbert_curve(lat, lon)

    return Row(hilbert_value=hilbert_value, timestamp=timestamp)

# clean up any rows with missing values
my_df_cleaned = my_df.filter(
    col("latitude").isNotNull() &
    col("longitude").isNotNull() &
    col("created_at").isNotNull()
)

# Map over the DataFrame to apply the Hilbert curve transformation
hilbert_rdd = my_df_cleaned.rdd.map(apply_hilbert_curve)

# Filter out None values resulting from invalid rows
hilbert_rdd = hilbert_rdd.filter(lambda row: row is not None)

# Check the first 5 rows to make sure the transformation is working
print("First 5 rows of hilbert_rdd:")
print(hilbert_rdd.take(5))

# Convert the RDD back to DataFrame with the appropriate schema
hilbert_schema = StructType([
    StructField("hilbert_value", IntegerType(), False),
    StructField("timestamp", TimestampType(), False)
])

hilbert_df = spark.createDataFrame(hilbert_rdd, hilbert_schema)

# Show the transformed data
hilbert_df.show(truncate=False)

# Save the result to a file
hilbert_df.write.option("header", "true").csv("/content/drive/MyDrive/hilbert_transformed_data3.csv")


First 5 rows of hilbert_rdd:
[Row(hilbert_value=20737, timestamp=datetime.datetime(2017, 7, 22, 9, 2, 53)), Row(hilbert_value=1041, timestamp=datetime.datetime(2017, 7, 22, 9, 2, 53)), Row(hilbert_value=17429, timestamp=datetime.datetime(2017, 7, 22, 9, 2, 53)), Row(hilbert_value=17409, timestamp=datetime.datetime(2017, 7, 22, 9, 2, 52)), Row(hilbert_value=20757, timestamp=datetime.datetime(2017, 7, 22, 9, 2, 53))]
+-------------+-------------------+
|hilbert_value|timestamp          |
+-------------+-------------------+
|20737        |2017-07-22 09:02:53|
|1041         |2017-07-22 09:02:53|
|17429        |2017-07-22 09:02:53|
|17409        |2017-07-22 09:02:52|
|20757        |2017-07-22 09:02:53|
|1297         |2017-07-22 09:02:53|
|4176         |2017-07-22 09:02:53|
|21761        |2017-07-22 09:02:53|
|20480        |2017-07-22 09:02:53|
|20544        |2017-07-22 09:02:53|
|16708        |2017-07-22 09:02:53|
|17409        |2017-07-22 09:02:53|
|1041         |2017-07-22 09:02:53|
|5396

In [27]:
SedonaRegistrator.registerAll(spark)


<ipython-input-27-1e2c0b20acd0>:1: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)


True

We spatially partitions the cleaned data using KDB-Tree partitioning to optimize spatial queries and improve performance. By analyzing the Spatial RDD, we ensure that the data is ready for efficient spatial operations. The KDB-Tree partitioning organizes the data into manageable sections based on spatial locality, reducing the search space during queries. Building an R-Tree index on each partition further enhances query efficiency by providing a hierarchical structure for rapid access to spatial data. Finally, converting the partitioned Spatial RDD back to a DataFrame allows for easier manipulation and analysis of the partitioned data using Spark's DataFrame API.






In [39]:
# 2.2

# Spatially partition the data using KDB-Tree partitioning
spatial_rdd = Adapter.toSpatialRdd(my_df_cleaned, "location")

# Analyze the spatial RDD
spatial_rdd.analyze()

# Perform spatial partitioning using KDB-Tree
spatial_rdd.spatialPartitioning(GridType.KDBTREE)

# Build an R-Tree index on each partition
spatial_rdd.buildIndex(IndexType.RTREE, True)

# Verify the number of partitions
num_partitions = spatial_rdd.rawSpatialRDD.getNumPartitions()
print(f"Number of spatial partitions: {num_partitions}")

partitioned_df = Adapter.toDf(spatial_rdd, spark)

Number of spatial partitions: 3


The spatial_temporal_range_query function performs a combined spatial and temporal query on a DataFrame containing spatial data. It begins by checking the data's range for latitude, longitude, and timestamps to ensure the query parameters are valid. The function then calculates grid-aligned coordinates and corresponding Hilbert values for the specified bounding box, which helps in efficiently filtering the data. It initially filters the DataFrame based on these Hilbert values and the specified time range, reducing the dataset size. A refinement step follows, joining the filtered results with the original DataFrame to ensure that only relevant spatial points within the exact bounds are included. Finally, the function outputs diagnostic information about the query's performance, including counts of points before and after refinement, aiding in understanding the effectiveness of the filtering process.

In [40]:
# 2.3
def spatial_temporal_range_query(spark, df, x1, y1, x2, y2, t1, t2, grid_size=256):
    import time
    start_time = time.time()

    # Print out some diagnostic information about the data
    print("Data range checks:")
    lat_range = df.agg(F.min('latitude'), F.max('latitude')).collect()[0]
    lon_range = df.agg(F.min('longitude'), F.max('longitude')).collect()[0]
    ts_range = df.agg(F.min('created_at'), F.max('created_at')).collect()[0]

    print(f"Latitude range: {lat_range[0]} to {lat_range[1]}")
    print(f"Longitude range: {lon_range[0]} to {lon_range[1]}")
    print(f"Timestamp range: {ts_range[0]} to {ts_range[1]}")

    # Use data's actual bounds if not specified
    min_lat = lat_range[0]
    max_lat = lat_range[1]
    min_lon = lon_range[0]
    max_lon = lon_range[1]

    # If no bounds provided, use full data range
    x1 = x1 if x1 is not None else min_lon
    y1 = y1 if y1 is not None else min_lat
    x2 = x2 if x2 is not None else max_lon
    y2 = y2 if y2 is not None else max_lat
    t1 = t1 if t1 is not None else str(ts_range[0])
    t2 = t2 if t2 is not None else str(ts_range[1])

    print(f"\nQuery Ranges:")
    print(f"Longitude: {x1} to {x2}")
    print(f"Latitude: {y1} to {y2}")
    print(f"Timestamp: {t1} to {t2}")

    # Expanded spatial-temporal approach
    query_df = my_df_cleaned.filter(
        (F.col('longitude').between(x1, x2)) &
        (F.col('latitude').between(y1, y2)) &
        (F.col('created_at').between(t1, t2))
    )

    #  using PySpark's spatial functions
    try:
        from pyspark.sql.functions import expr
        query_df = query_df.filter(
            expr(f"ST_Contains(ST_PolygonFromEnvelope({x1}, {y1}, {x2}, {y2}), location)")
        )
    except:
        print("ST_Contains not available, using standard filtering")

    # Performance and result reporting
    end_time = time.time()
    total_results = query_df.count()

    print(f"\nQuery Results:")
    print(f"Total points found: {total_results}")
    print(f"Query execution time: {end_time - start_time:.4f} seconds")

    return query_df

# Example queries with different ranges
# Wide range query
result_df = spatial_temporal_range_query(
    spark,
    my_df_cleaned,
    x1=-2.3,    # Minimum longitude
    y1=53.3,    # Minimum latitude
    x2=-2.1,    # Maximum longitude
    y2=53.6,    # Maximum latitude
    t1="2017-07-22 00:00:00",  # Start time
    t2="2017-07-22 23:59:59"   # End time
)

# Show results
result_df.show(10, truncate=False)
print("\nTotal results:", result_df.count())

#  Visualize the distribution
from pyspark.sql.functions import col
distribution = result_df.groupBy(
    F.floor(col("longitude") * 10) / 10,
    F.floor(col("latitude") * 10) / 10
).count().orderBy("count", ascending=False)

print("\nSpatial distribution:")
distribution.show(10)

Data range checks:
Latitude range: -89.43579825 to 78.62645647
Longitude range: -178.72606449 to 178.45017672
Timestamp range: 2017-07-22 09:02:52 to 2017-07-22 09:48:54

Query Ranges:
Longitude: -2.3 to -2.1
Latitude: 53.3 to 53.6
Timestamp: 2017-07-22 00:00:00 to 2017-07-22 23:59:59

Query Results:
Total points found: 320
Query execution time: 17.4975 seconds
+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+------------------+------------------------------------+
|created_at         |text                                                                                                                                        |longitude |latitude          |location                            |
+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------+----------+---